INGESTA Y LIMPIEZA DEL DATASET

Transformaciones: El analista de datos requiere estas, y solo estas, transformaciones para sus datos:

Generar campo id: Cada id se compondrá de la primera letra del nombre de la plataforma, seguido del show_id ya presente en los datasets (ejemplo para títulos de Amazon = as123)

Los valores nulos del campo rating deberán reemplazarse por el string “G” (corresponde al maturity rating: “general for all audiences”

De haber fechas, deberán tener el formato AAAA-mm-dd

Los campos de texto deberán estar en minúsculas, sin excepciones

El campo duration debe convertirse en dos campos: duration_int y duration_type. El primero será un integer y el segundo un string indicando la unidad de medición de duración: min (minutos) o season (temporadas)

In [30]:
import pandas as pd

In [31]:
# IMPORTO LOS DATASETS CON LOS CUALES VOY A TRABAJAR Y LOS GUARDO A CADA UNO EN SU PROPIO DATAFRAME DE PANDAS.
df_a = pd.read_csv('./Datasets/amazon_prime_titles-score.csv')
df_d = pd.read_csv('./Datasets/disney_plus_titles-score.csv')
df_h = pd.read_csv('./Datasets/hulu_titles-score (2).csv')
df_n = pd.read_csv('./Datasets/netflix_titles-score.csv')

In [32]:
#GENERACION DE LOS CAMPOS ID PARA CADA DATAFRAME

df_a.insert(0, 'id', df_a['show_id'], allow_duplicates=False)
df_a['id'] = 'a' + df_a['show_id']

df_d.insert(0, 'id', df_d['show_id'], allow_duplicates=False)
df_d['id'] = 'd' + df_d['show_id']

df_h.insert(0, 'id', df_h['show_id'], allow_duplicates=False)
df_h['id'] = 'h' + df_h['show_id']

df_n.insert(0, 'id', df_n['show_id'], allow_duplicates=False)
df_n['id'] = 'n' + df_n['show_id']

In [33]:
#GENERO UNA COLUMNA CON EL NOMBRE DE CADA PLATAFORMA
df_a['plataforma'] = 'Amazon'
df_d['plataforma'] = 'Disney'
df_h['plataforma'] = 'Hulu'
df_n['plataforma'] = 'Netflix'

In [34]:
#GENERO UN DATAFRAME UNIFICADO CON LOS CUATRO DATAFRAMES

df = pd.concat([df_a, df_d, df_h, df_n],ignore_index=True)

In [35]:
#REEMPLAZO DE VALORES SOBRE EL CAMPO RATING
df['rating'] = df['rating'].fillna('G')

In [36]:
#CAMBIO DE FORMATO DE FECHAS
df['date_added'] = pd.to_datetime(df['date_added'])


In [37]:
#CAMBIO DE FORMATO DE CAMPOS DE TEXTO
for col in list(df.columns):
  try:
    df[col] = df[col].str.lower()
  except (TypeError,AttributeError):
    pass

In [38]:
#OPERACIONES SOBRE EL CAMPO DURATION
a = df['duration'].str.split(expand=True) #Creo un Dataframe secundario con el contenido de la columna duration separado en dos columnas.
a.rename(columns={0:'duration_int', 1:'duration_type'}, inplace=True)   #Este paso en este momento es redundante, pero para mejorar el entendimiento del 
                                                                        #código cambio los nombres de las columnas ahora.

In [39]:
#Veo que el campo 'duration_int' presenta valores nulos.
a['duration_int'].info()  

<class 'pandas.core.series.Series'>
RangeIndex: 22998 entries, 0 to 22997
Series name: duration_int
Non-Null Count  Dtype 
--------------  ----- 
22516 non-null  object
dtypes: object(1)
memory usage: 179.8+ KB


In [40]:
#Reemplazo los valore nulos de el campo 'duration_int' por el valor '0' (teniendo en cuenta que la columna contiene datos del tipo string).

a['duration_int'].fillna('0', inplace = True)

In [41]:
#Cambio el Type de la variable 'duration_int' a integer.
a['duration_int'] = a['duration_int'].astype(int)

In [42]:
#Veo que en el campo 'duration_type' tengo 4 tipos de valores.
a['duration_type'].unique()

array(['min', 'season', 'seasons', nan], dtype=object)

In [43]:
#Unifico los campos 'seasons' y 'season' bajo el valor 'season'.
a['duration_type'].loc[a['duration_type']=='seasons']='season'

C:\Users\VAIO\AppData\Local\Temp\ipykernel_9464\4087776168.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  a['duration_type'].loc[a['duration_type']=='seasons']='season'


In [44]:
#En el DF pricipal, veo que las columnas pueden ser del tipo 'movie' (que tiene 'duration_type' en min) o tipo 'tv show' (que tiene 'duration_type' con valores season/seasons).
df['type'].unique()

array(['movie', 'tv show'], dtype=object)

In [45]:
#Creo las columnas en el Dataframe con los valores de el Dataframe secundario.
df['duration_int'] = a['duration_int']
df['duration_type'] = a['duration_type']

In [46]:
df.head(2)

,id,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,score,plataforma,duration_int,duration_type
0,as1,s1,movie,the grand seduction,don mckellar,"brendan gleeson, taylor kitsch, gordon pinsent",canada,2021-03-30,2014,g,113 min,"comedy, drama",a small fishing village must procure a local d...,99,amazon,113,min
1,as2,s2,movie,take care good night,girish joshi,"mahesh manjrekar, abhay mahajan, sachin khedekar",india,2021-03-30,2018,13+,110 min,"drama, international",a metro family decides to fight a cyber crimin...,37,amazon,110,min


In [47]:
#GUARDO EL DATAFRAME EN UN ARCHIVO CSV
df.to_csv('data_pi.csv',index = False)